# Newcastle (w/ Value Constraints)


### Load and downsample constraints

First, we load the interpretations from our digital outcrop model (DOM). These are spaced every ~1 cm, which is unnecessarily high resolution for our 3D model, so we also downsample them using `scipy.kdtree`.

In [1]:
import curlew
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

In [2]:
import torch
curlew.dtype = torch.float64 # smaller coordinates require higher precision
curlew.device = "cuda"

In [3]:
# Plotting params
from matplotlib import colors
import matplotlib as mpl
import matplotlib.font_manager as font_manager

font_dirs = ['../Helvet/',]
font_files = font_manager.findSystemFonts(fontpaths=font_dirs[0], fontext="ttf")

for font_file in font_files:
    font_manager.fontManager.addfont(font_file)

# Plotting params
# RCParams
import matplotlib as mpl
mpl.rcParams["text.usetex"] = False
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = 'Helvetica'
plt.rcParams["mathtext.fontset"] = "stixsans"
plt.rcParams["mathtext.default"] = "it"
plt.rcParams['xtick.labelsize'] = 18
plt.rcParams['ytick.labelsize'] = 18
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['axes.formatter.use_mathtext'] = True
plt.rcParams['font.size'] = 18
plt.rcParams["axes.formatter.limits"] = [-2, 2]

# Curlew colormap
import matplotlib.colors as mcolors

# Define the colors extracted manually from the provided logo image
colors = [
    "#A6340B",  # rich red (not darkest)
    "#E35B0E",  # vibrant orange-red
    "#F39C12",  # medium orange
    "#F0C419",  # bright orange-yellow
    "#FAE8B6",  # soft pale orange (close to white but not pure white)
    "#8CD9E0",  # light cyan blue
    "#31B4C2",  # medium cyan-blue
    "#1B768F",  # medium blue 
    "#054862",  # deeper blue (not darkest)
]

# Create a discrete colormap using these colors
curlew_cmap = mcolors.ListedColormap(colors)

# Random seed
random.seed(42)

### Load planar orientation ("compass") measurements

In [4]:
planes = pd.read_csv('traces/newcastle_planes.csv')
planes['Unit'] = [ t.split('.interpretation.')[1].split('.')[0] for t in planes['Name'] ] # shorten names so they are less ugly

for k in ['Sample_Radius', 'RMS', 'Gx', 'Gy', 'Gz', 'Length', 'Name']: # delete unwanted columns
    del planes[k]

### Load and downsample structure normal (SNE) estimates

Combine with the above "compass" measurements to our value constraint arrays

In [5]:
from curlew.io import loadPLY
fault = loadPLY('traces/FaultSNE.ply')
dykeL = loadPLY('traces/DykeBottom.ply')
dykeU = loadPLY('traces/DykeTop.ply')

from scipy.spatial import KDTree
scale = 0.5 # average points within this many meters
for s in [fault, dykeL, dykeU]:
    tree = KDTree( s['xyz'] ) # build KD-tree object
    collapsed = np.full( (len(s['xyz']),), False ) # init array used to track which points have already been averaged
    # loop through points and do averaging / collapsing
    points = []
    normals = []
    for j in range(len(s['xyz'])):
        if collapsed[j]:
            continue # ignore point - it has already been collapsed!
        else:

            # get neighbourhood points
            idx = tree.query_ball_point(s['xyz'][ j, : ], r = scale )

            # average point location - this is our subsampled position
            points.append( np.mean( s['xyz'][idx, : ], axis=0 ) ) # average position vector
            normals.append( np.mean( s['normals'][idx, : ], axis=0 ) ) # average normal vector
            normals[-1] = normals[-1] / np.linalg.norm(normals[-1]) # normalise to length 1

            # flag points that are already moved to the output (condensed)
            collapsed[idx] = True
    s['xyz'] = np.array(points)
    s['normals'] = np.array(normals)
    if 'attr' in s: del s['attr'] # not needed


In [6]:
gradient_constraints = {
    'Dyke' : ( np.vstack( [dykeU['xyz'], dykeL['xyz']] ), np.vstack( [dykeU['normals'], dykeL['normals']] ) ),
    'Fault' : (fault['xyz'], fault['normals']),
}

for u in np.unique( planes['Unit'] ):
    xyz = np.array( planes[ planes['Unit'] == u][['Cx','Cy','Cz']] ) # position
    klm = np.array( planes[ planes['Unit'] == u][['Nx','Ny','Nz']] ) # normal
    if u in gradient_constraints:
        gradient_constraints[u] = (np.vstack([ gradient_constraints[u][0], xyz]), np.vstack([ gradient_constraints[u][1], klm]) )
    else:
        gradient_constraints[u] = (xyz, klm)

In [7]:
# ensure dyke gradient constraints point in the right direction
gradient_constraints['Dyke'][1][ gradient_constraints['Dyke'][1][:,0] > 0 ] *= -1

Load contact traces, downsample them and convert to value constraints.

In [8]:
traces = pd.read_csv('traces/newcastle_traces.csv')
traces['Unit'] = [ t.split('.interpretation.')[1].replace(' Boundary','').replace('.Trace','') for t in traces['Name'] ] # shorten names so they are less ugly

for k in ['Name','Point_id', 'Cost', 'Cost_Mode']: # delete unwanted columns
  del traces[k]

unit_codes = {'Dyke.Upper' : 0.25, 'Dyke.Lower':-0.25, # N.B. dyke is ~0.5 m thick
        'TopBarBeachShales' : 22.2, # in this case we use the approx meters above sea level as the scalar value, as these are relatively flat beds
        'TopDudleyCoal' : 21.2,
        'TopBogeyHole' : 18.,
        'Fault' : 0., 
        # N.B. all other Unit names in the traces file will be ignored
        }


# loop through each lithology type and subsample contact points
from scipy.spatial import KDTree
value_constraints = {}
for n,sv in unit_codes.items():
  # find points of this lithology
  mask = np.array(traces['Unit']) == n # which points are in this lithology?
  c = np.sum(mask) # count points in this lithology
  print("Filtering %d points for %s ... " %(c, n ), end='' )

  # construct a KD tree
  xyz = np.array([traces[k] for k in ['Start_x', 'Start_y', 'Start_z']]).T
  _xyz = xyz[mask, : ] # subset main point array
  tree = KDTree( _xyz ) # build KD-tree object

  # init array used to track which points have already been collapsed
  collapsed = np.full( (c,), False )

  # loop through points and do averaging / collapsing
  points = []
  for j in range(c):
      if collapsed[j]:
        continue # ignore point - it has already been collapsed!
      else:

        # get neighbourhood points
        idx = tree.query_ball_point(_xyz[ j, : ], r = scale )

        # average point location - this is our subsampled position
        points.append( np.mean( _xyz[idx, : ], axis=0 ) )

        # flag points that are already moved to the output (condensed)
        collapsed[idx] = True
  
  print("%d points remain!" % len(points) )
  value_constraints[ n ] = np.array( points ), np.array( [sv] * len( points ) )

Filtering 826 points for Dyke.Upper ... 20 points remain!
Filtering 903 points for Dyke.Lower ... 21 points remain!
Filtering 514 points for TopBarBeachShales ... 13 points remain!
Filtering 463 points for TopDudleyCoal ... 11 points remain!
Filtering 1080 points for TopBogeyHole ... 26 points remain!
Filtering 906 points for Fault ... 20 points remain!


Export constraints to PLY for QAQC

In [9]:
from curlew.io import savePLY

for k, (p,v) in value_constraints.items():
    savePLY('constraints/%s.ply'%k, xyz=p, attr=v[:,None])

Load point cloud (of outcrop surface) and use it to compute model extent.

In [21]:
outcrop = loadPLY('./Newcastle_5cm.ply')
xyz = outcrop['xyz']
extents = np.array( [np.percentile(xyz[:,i], (0,100)) for i in range(3)] ) # extent of outcrop points
dims = [ (e[1] - e[0]) for e in extents] # dims to get an 0.5 m grid

# create grid for global constraints
from curlew.geometry import grid
grid_obj = grid(dims, (1,1,1), origin=extents[:,0])
cg = grid_obj.coords()
grid_obj.sampleArgs = dict(N=512)
savePLY('./grid.ply', xyz=cg ) # save for qaqc

#### Build a model

Now combine the above constraints to build our interpolation!

In [22]:
from curlew import HSet
from curlew import CSet
from curlew.geology import strati, sheet, fault
from curlew.geology.model import GeoModel

# define shared hyperparamets first
H = HSet( value_loss=1.0, grad_loss=1.0, mono_loss="0.1", thick_loss='0.1')
params = dict(
    input_dim=3, # 3D model
    rff_features=64, # number of fourier features
    learning_rate=1e-3
)

# STRATIGRAPHY
C0 = CSet( vp=np.vstack( [value_constraints[k][0] for k in ['TopBarBeachShales','TopDudleyCoal', 'TopBogeyHole']] ), # stratigraphic value constraint positions
           vv=np.hstack( [value_constraints[k][1] for k in ['TopBarBeachShales','TopDudleyCoal', 'TopBogeyHole']] ), # stratigraphic value constraint values
           gp=np.vstack( [gradient_constraints[k][0] for k in ['TopBogeyHole', 'Bedding']] ), # stratigraphic gradient constraint positions
           gv=np.vstack( [gradient_constraints[k][1] for k in ['TopBogeyHole', 'Bedding']] ), # stratigraphic gradient constraint vectors
           grid=grid_obj, # where to evaluate "global" constraints
           delta=1 )# grid spacing
C0.trend = np.mean( C0.gv, axis=0 ) / np.linalg.norm( np.mean( C0.gv, axis=0 ) ) # specify global trend to encourage flatness

s0 = strati('sediments', # interpolator for host rock field
            C0, # constraints
            H, # hyperparameters
            length_scales=[20, 100], # frequencies to use for this structure]
            hidden_layers=[8],
            **params)
# FAULT
C1 = CSet( vp=np.vstack( [value_constraints[k][0] for k in ['Fault']] ), # stratigraphic value constraint positions
           vv=np.hstack( [value_constraints[k][1] for k in ['Fault']] ), # stratigraphic value constraint values
           gp=np.vstack( [gradient_constraints[k][0] for k in ['Fault']] ), # stratigraphic gradient constraint positions
           gv=np.vstack( [gradient_constraints[k][1] for k in ['Fault']] ), # stratigraphic gradient constraint vectors
           grid=grid_obj, # where to evaluate "global" constraints
           delta=1) # grid spacing

s1 = fault('fault', # interpolator for host rock field
            C1, # constraints
            H, # hyperparameters
            sigma1=(0,0,1),
            offset=(1.0, 0.0, 1.0),
            length_scales=[10, 100], # frequencies to use for this structure
            hidden_layers=[8],
            **params)


# DYKE
C2 = CSet( vp=np.vstack( [value_constraints[k][0] for k in ['Dyke.Upper','Dyke.Lower']] ), # stratigraphic value constraint positions
           vv=np.hstack( [value_constraints[k][1] for k in ['Dyke.Upper','Dyke.Lower']] ), # stratigraphic value constraint values
           gp=np.vstack( [gradient_constraints[k][0] for k in ['Dyke']] ), # gradient orientation constraints
           gv=np.vstack( [gradient_constraints[k][1] for k in ['Dyke']] ), # gradient orientation constraints
           grid=grid_obj, # where to evaluate "global" constraints
           delta=1 ) # grid spacing

s2 = sheet('dyke', # interpolator for host rock field
            C2, # constraints
            H.copy(thick_loss="0.01", mono_loss="1.0"), # value loss is important here!
            length_scales=[3,100], # frequencies to use for this structure
            contact=(value_constraints['Dyke.Lower'][0], value_constraints['Dyke.Upper'][0]), # scalar values representing the top and bottom of the dyke
            hidden_layers=[32],
            **params)

In [23]:
for k in ['TopBarBeachShales','TopDudleyCoal', 'TopBogeyHole']:
    s0.addIsosurface(name=k, seed=value_constraints[k][0])
for k in ["Dyke.Upper", "Dyke.Lower"]:
    s2.addIsosurface(name=k, seed=value_constraints[k][0])

In [24]:
# combine into a geomodel
M = GeoModel([s0, s1, s2]) # s1
loss = M.prefit( epochs=300 ) # fit scalar fields independently

dyke: 300/300|, value_loss=0.00801, grad_loss=0.00592, thick_loss=0.00971, mono_loss=0.0342
Evaluating: 100%|██████████| 1/1 [00:00<00:00, 223.35it/s]
fault: 300/300|, value_loss=0.000162, grad_loss=0.0638, thick_loss=0.000181, mono_loss=0.000288
Evaluating: 100%|██████████| 1/1 [00:00<00:00, 181.34it/s]
sediments: 300/300|, value_loss=0.167, grad_loss=0.175, thick_loss=0.00433, mono_loss=0.00418, flat_loss=0.0147


In [25]:
# freeze fault and optimize fault offset
if True:
    M.freeze([s1, s2]) 
    M.fit( 1000, learning_rate=1e-1 )
    print("Optimised fault offset is: %.2f" % s1.field.offset.item())

Training: 1000/1000|, dyke=0.0562, fault=0.065, sediments=0.047 

Optimised fault offset is: 2.13


In [26]:
# evaluate model on outcrop surface
pred = M.predict(outcrop['xyz'])
# savePLY('./outcrop.ply', xyz=outcrop['xyz'], attr=pred ) # save for qaqc

In [28]:
# evaluate model on a grid with 20 cm spacing
fine_grid_obj = grid(dims, (0.2,0.2,0.2), origin=extents[:,0])
eg = fine_grid_obj.coords()
pred = M.predict(eg)
# savePLY('./model.ply', xyz=eg, attr=pred ) # save for qaqc

In [29]:
# Load the HD outcrop
hd_outcrop = loadPLY("Newcastle_1cm.ply")
hd_xyz = hd_outcrop["xyz"]

In [30]:
# Predict the classes for the HD Outcrop
hd_classes = M.classify(hd_xyz)[0][:, 0]

In [31]:
# Normalise and colormap
norm = mcolors.Normalize(vmin=hd_classes.min(), vmax=hd_classes.max())
hd_rgba = curlew_cmap(norm(hd_classes))

In [32]:
# Alpha compositing
def alpha_composite(fg, bg):
    """
    Composites foreground RGBA colors over background RGBA colors.
    
    Parameters
    ----------
    fg : ndarray of shape (N, 4)
        Foreground colors (RGBA), with values in [0, 1]
    bg : ndarray of shape (N, 4)
        Background colors (RGBA), with values in [0, 1]
    
    Returns
    -------
    out : ndarray of shape (N, 4)
        Composited colors (RGBA)
    """
    fg = np.asarray(fg)
    bg = np.asarray(bg)

    fg_rgb = fg[:, :3]
    fg_alpha = fg[:, 3:4]
    bg_rgb = bg[:, :3]
    bg_alpha = bg[:, 3:4]

    out_rgb = fg_rgb * fg_alpha + bg_rgb * (1 - fg_alpha)
    out_alpha = fg_alpha + bg_alpha * (1 - fg_alpha)

    out = np.concatenate([out_rgb, out_alpha], axis=1)
    return out

In [33]:
# Composite RGB
bg = hd_outcrop["rgb"]/255
bg = np.c_[bg, np.ones_like(bg[:, 0])]
fg = np.c_[hd_rgba[:, :-1], 0.5 * np.ones_like(hd_rgba[:, 0])]
comp_rgba = alpha_composite(fg, bg)

In [34]:
# Save the outcrop
savePLY("./outcrop_classes.ply", xyz=hd_xyz, rgb=(comp_rgba[:, :-1]*255).astype(int))

In [35]:
# Predict on the HD Outcrop
predicted_sf_id = M.predict(hd_xyz)

In [36]:
# Exclude the dyke
non_dyke = predicted_sf_id[:, 1] > 1
non_dyke_xyz = hd_xyz[non_dyke]
non_dyke_classes = hd_classes[non_dyke]

# Undeform the xyz
undeformed_xyz = s1.undeform(non_dyke_xyz)

In [37]:
# Save the undeformed point cloud
savePLY("./outcrop_classes_undeformed.ply", xyz=undeformed_xyz, rgb=(comp_rgba[non_dyke, :-1]*255).astype(int))